# Preparing datasets

In [1]:
# %config InlineBackend.figure_format='retina'
from ekorpkit import eKonf

eKonf.setLogger("INFO")
print("version:", eKonf.__version__)
print("is notebook?", eKonf.is_notebook())
print("is colab?", eKonf.is_colab())
print("evironment varialbles:")
eKonf.print(eKonf.env().dict())

INFO:ekorpkit.base:IPython version: (6, 9, 0), client: jupyter_client
INFO:ekorpkit.base:Google Colab not detected.


version: 0.1.35+3.g5f275b3.dirty
is notebook? True
is colab? False
evironment varialbles:
{'CUDA_DEVICE_ORDER': None,
 'CUDA_VISIBLE_DEVICES': None,
 'EKORPKIT_CONFIG_DIR': '/workspace/projects/ekorpkit-book/config',
 'EKORPKIT_DATA_DIR': None,
 'EKORPKIT_LOG_LEVEL': 'INFO',
 'EKORPKIT_PROJECT': 'ekorpkit-book',
 'EKORPKIT_WORKSPACE_ROOT': '/workspace',
 'KMP_DUPLICATE_LIB_OK': 'TRUE',
 'NUM_WORKERS': 230}


In [2]:
cfg = eKonf.compose("io/fetcher=labelstudio")
cfg.project_id = 2
ls = eKonf.instantiate(cfg)

INFO:ekorpkit.base:IPython version: (6, 9, 0), client: jupyter_client
INFO:ekorpkit.io.fetch.base:/workspace/.tmp/fetcher/annotations.json already exists. skipping..


In [10]:
import pandas as pd

df = pd.read_json(cfg.path.output.filepath)
df

,id,annotations,file_upload,drafts,predictions,data,meta,created_at,updated_at,project
0,26444,"[{'id': 17465, 'completed_by': 6, 'result': [{...",4762eef4-esg_topics2_sample_for_label.csv,[],[],"{'corpus': 'modu_news', 'id': 'NIRW1900000018....",{},2022-03-10 00:22:48.222340+00:00,2022-07-06 16:27:23.053585+00:00,2
1,26377,"[{'id': 17449, 'completed_by': 6, 'result': [{...",4762eef4-esg_topics2_sample_for_label.csv,[],[],"{'corpus': 'modu_news', 'id': 'NPRW1900000057....",{},2022-03-10 00:22:48.219516+00:00,2022-07-05 15:19:36.531306+00:00,2
2,26426,"[{'id': 17225, 'completed_by': 7, 'result': [{...",4762eef4-esg_topics2_sample_for_label.csv,[],[],"{'corpus': 'modu_news', 'id': 'NWRW1900000035....",{},2022-03-10 00:22:48.221577+00:00,2022-07-04 11:25:45.115647+00:00,2
3,26389,"[{'id': 17222, 'completed_by': 7, 'result': [{...",4762eef4-esg_topics2_sample_for_label.csv,[],[],"{'corpus': 'modu_news', 'id': 'NWRW1900000005....",{},2022-03-10 00:22:48.220021+00:00,2022-07-04 11:25:13.301604+00:00,2
4,26388,"[{'id': 17221, 'completed_by': 7, 'result': [{...",4762eef4-esg_topics2_sample_for_label.csv,[],[],"{'corpus': 'modu_news', 'id': 'NWRW1900000005....",{},2022-03-10 00:22:48.219978+00:00,2022-07-04 11:24:50.585600+00:00,2
...,...,...,...,...,...,...,...,...,...,...
13615,20371,"[{'id': 147, 'completed_by': 2, 'result': [{'v...",4762eef4-esg_topics2_sample_for_label.csv,[],[],"{'corpus': 'esg_report', 'id': 'ESG000230.0009...",{},2022-03-10 00:22:47.803701+00:00,2022-03-10 00:22:47.803710+00:00,2
13616,20370,"[{'id': 1412, 'completed_by': 4, 'result': [{'...",4762eef4-esg_topics2_sample_for_label.csv,[],[],"{'corpus': 'esg_report', 'id': 'ESG000230.0009...",{},2022-03-10 00:22:47.803659+00:00,2022-03-10 00:22:47.803668+00:00,2
13617,20369,"[{'id': 1411, 'completed_by': 4, 'result': [{'...",4762eef4-esg_topics2_sample_for_label.csv,[],[],"{'corpus': 'esg_report', 'id': 'ESG000230.0009...",{},2022-03-10 00:22:47.803614+00:00,2022-03-10 00:22:47.803624+00:00,2
13618,20367,"[{'id': 1409, 'completed_by': 4, 'result': [{'...",4762eef4-esg_topics2_sample_for_label.csv,"[{'id': 597, 'user': 'pumpkinhead@hanyang.ac.k...",[],"{'corpus': 'esg_report', 'id': 'ESG000710.0013...",{},2022-03-10 00:22:47.803523+00:00,2022-03-10 00:22:47.803533+00:00,2


In [9]:
df.annotations[300]

[{'id': 2884,
  'completed_by': 6,
  'result': [{'value': {'choices': ['Negative']},
    'id': 'huJhd4sFaG',
    'from_name': 'sentiment',
    'to_name': 'text',
    'type': 'choices',
    'origin': 'manual'}],
  'was_cancelled': False,
  'ground_truth': False,
  'created_at': '2022-06-03T04:18:17.257102Z',
  'updated_at': '2022-06-03T04:18:34.766446Z',
  'lead_time': 6673.49,
  'prediction': {},
  'result_count': 0,
  'task': 21262,
  'parent_prediction': None,
  'parent_annotation': None},
 {'id': 8215,
  'completed_by': 8,
  'result': [{'value': {'choices': ['Negative']},
    'id': 'onvJoWr3VO',
    'from_name': 'sentiment',
    'to_name': 'text',
    'type': 'choices',
    'origin': 'manual'}],
  'was_cancelled': False,
  'ground_truth': False,
  'created_at': '2022-06-21T03:13:48.040895Z',
  'updated_at': '2022-06-21T03:13:48.040916Z',
  'lead_time': 0.968,
  'prediction': {},
  'result_count': 0,
  'task': 21262,
  'parent_prediction': None,
  'parent_annotation': None}]

In [11]:
df.data[300]

{'corpus': 'esg_report',
 'id': 'ESG001110.0030',
 'topic_num': 'topic11',
 'topic_weight': 0.9966742396354671,
 'top_topics': 'G(지배구조)[100%], Topic #20[0%], S(사회적 가치)[0%], S(일자리)[0%], S(협력사)[0%]',
 'chunk_id': 4,
 'text': '등기임원일지라도 이사회에 참석하지 않고 경영상 중요한 영향을 미치는 안건들에 대해 검토 결의하지 않는다면 이사로서의 역할을 제대로 수행하지 못한 것이므로 실질적인 책임경영을 하고 있다고 볼 수 없음\n본 보고서에서는 총수일가가 이사로 등재되어 있는 기업 중 사업보고서 공시대상인 79사 를 대상으로 2018년부터 2019년 1분기 까지의 이사회 참석률을 살펴봄\n우선 기업별로 살펴보면, 조사대상인 79사 중 총수일가의 이사회 참석률이 100%인 기업은 26사이며, 반면 총수일가의 이사회 참석률이 10% 미만인 기업은 18사 존재함'}